Final paper: Course on Digital Satellite Image Processing

*Author:* Veronica Andreo

*Date:* 2022-06-08-08

This notebook will demonstrate the use of GRASS GIS 8.2+ in combination with Python within a Jupyter Notebook environment. We will use GRASS modules and python libraries that are part of the distribution that facilitate scripting (`grass.script`) and connecting and interacting with Jupyter Notebooks (`grass.jupyter`). 

In order to run this notebook, the development version of GRASS GIS is required, as well as all its dependencies and the `sentinelsat` library. 

The workflow that will be demonstrated on this notebook ranges from searching satellite data to classifying it using machine learning. 

## GRASS GIS basics


### Open GRASS for the first time

As of version 8 GRASS has modified its startup to make it more user friendly

![First time launching GRASS 8](https://grass.osgeo.org/grass-stable/manuals/grass_start.png)

From the *Data* catalog tab you can manage several actions and if you do not yet have imported data in the GRASS database, the software creates the directory structure automatically


### Database

- **GRASS database** (directory with projects): When running GRASS GIS for the first time, a folder named "grassdata" is automatically created. Depending on the operating system, it can be found in `$HOME` (*nix) or `My Documents` (MS Windows).
- **Location** (a project): A "location" is defined by its coordinate reference system (CRS). The location that is automatically created is in WGS84 (EPSG:4326). If you have data in another CRS, you should ideally create a new location
- **Mapset** (a subproject): Each location can have many "mapsets" to manage different aspects of a project or sub-regions of a project. When creating a new Location, GRASS GIS automatically creates a special Mapset called *PERMANENT* where the central project data (e.g., base maps, road network, dem, etc) can be stored. 
    
![GRASS GIS database](https://grass.osgeo.org/grass-stable/manuals/grass_database.png)

More info: https://grass.osgeo.org/grass-stable/manuals/grass_database.html


### Computational region

Another fundamental concept of GRASS GIS (and very useful when working with raster data) is that of the **computational region**. This refers to the boundary configuration of the analysis area and spatial resolution (raster). The **computational region** can be defined and modified with the command [g.region](https://grass.osgeo.org/grass-stable/manuals/g.region.html) to the extent of a vector map, a raster or manually to some area of interest. The output raster maps *(output)* will have an extent and spatial resolution equal to the computational region, while vector maps are always processed at their original extent

![Computational region](https://gitlab.com/veroandreo/maie-procesamiento/-/raw/taller-grass-online/assets/img/region.png)

For more details see the wiki on [Computational Region](https://grasswiki.osgeo.org/wiki/Computational_region)

### Modules and extensions

More than [500 modules](https://grass.osgeo.org/grass-stable/manuals/full_index.html) for the most varied tasks, but with a clear organization:

| Prefix                                                               | Function class   | Type of command                     | Example
|--------------------------------------------------------------------- |:---------------- |:----------------------------------- |:-------------------------------------------------------------------------------------------------------------------
| [g.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#g)    | general          | general data management             | [g.rename](https://grass.osgeo.org/grass-stable/manuals/g.rename.html): renames map
| [d.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#d)    | display          | graphical output                    | [d.rast](https://grass.osgeo.org/grass-stable/manuals/d.rast.html): display raster map 
| [r.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r)    | raster           | raster processing                   | [r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html): map algebra
| [v.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r)    | vector           | vector processing                   | [v.clean](https://grass.osgeo.org/grass-stable/manuals/v.clean.html): topological cleaning
| [i.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#i)    | imagery          | imagery processing                  | [i.pca](https://grass.osgeo.org/grass-stable/manuals/i.pca.html): Principal Components Analysis on imagery group
| [r3.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r3)  | voxel            | 3D raster processing                | [r3.stats](https://grass.osgeo.org/grass-stable/manuals/r3.stats.html): voxel statistics
| [db.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#db)  | database         | database management                 | [db.select](https://grass.osgeo.org/grass-stable/manuals/db.select.html): select value(s) from table
| [ps.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#ps)  | postscript       | PostScript map creation             | [ps.map](https://grass.osgeo.org/grass-stable/manuals/ps.map.html): PostScript map creation
| [t.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#t)    | temporal         | space-time datasets                 | [t.rast.aggregate](https://grass.osgeo.org/grass-stable/manuals/t.rast.aggregate.html): raster time series aggregation

Extensions or **add-ons** can be installed from the
[central repository](https://grass.osgeo.org/grass-stable/manuals/addons/) 
or from *GitHub* (or other similar ones) using the command 
[g.extension](https://grass.osgeo.org/grass-stable/manuals/g.extension.html)

```bash
 # install an extension from the GRASS GIS repository
 g.extension extension=r.hants
 
 # install an extension from another GitHub repository
 g.extension extension=r.in.sos \
   url=https://github.com/pesekon2/GRASS-GIS-SOS-tools/tree/master/sos/r.in.sos
``` 


## GRASS & Python

### grass.script

The **grass.script** or GRASS GIS Python Scripting Library provides functions for calling GRASS modules within Python scripts as threads. The most commonly used functions include:

- `run_command`: used when the output of the modules is a raster or vector, no text type output is expected
- `read_command`: used when the output of the modules is a text
- `parse_command`: used with modules whose output can be converted to `key=value` pairs
- `write_command`: used with modules that expect text input, either in the form of a file or from stdin

It also provides several wrapper functions for frequently used modules:

- To get info from a raster, script.raster.raster_info() is used: `gs.raster_info('dsm')`
- To get info of a vector, script.vector.vector_info() is used: `gs.vector_info('roads')`
- To list the raster in a location, script.core.list_grouped() is used: `gs.list_grouped(type=['raster'])`
- To obtain the computational region, script.core.region() is used: `gs.region()`

More info: https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html
### grass.jupyter
### grass.jupyter

The **grass.jupyter** library improves the integration of GRASS and Jupyter, and provides different classes to visualize maps.

- `init`: starts a GRASS session and sets up necessary environment variables
- `Map`: 2D rendering
- `Map3D`: 3D rendering
- `InteractiveMap`: interactive visualization with folium
- `TimeSeriesMap`: visualization for spatio-temporal data

More info: https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html


## 1. Paths and variables

In [ ]:
# Paths
s2_credentials="/home/veroandreo/gisdata/SENTINEL_SETTING.txt"
s2_data_folder="/home/veroandreo/gisdata/s2_data"
train_areas="training.gpkg"
grassdata="../grassdata"

# Variables
location="posgar2007_4_cba"
mapset="PERMANENT"
start_date="2022-03-01"
end_date="2022-04-30"

## 2. Imports and initialization of GRASS GIS

In [ ]:
import os
import subprocess
import sys
import pandas as pd

# Ask GRASS GIS where its Python packages are to be able to start it from the notebook
sys.path.append(
    subprocess.check_output(["grassdev", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init(grassdata, location, mapset)

The location to be used for this notebook was prepared for the 2020 edition of MAIE's Digital Satellite Image Processing and GIS course. More details and download links can be found in the course [repo](https://gitlab.com/veroandreo/maie-procesamiento#datos).

Before we start, we list the elements in the mapset `PERMANENT`. If we only want to see the raster or vector type elements, just change the `type` option in the following command. As you can see, since the output is of text type, we use `read_command()`.

In [ ]:
# List all elements in the PERMANENT mapset
print(gs.read_command("g.list", type="all", mapset="."))

Next, we create a new mapset or project, where we will work with this notebook and import Sentinel-2 data.

In [ ]:
# Create a new mapset
gs.run_command("g.mapset", mapset="sentinel2", flags="c")

In [ ]:
# Print accessible mapsets within the location
print(gs.read_command("g.mapsets", flags="p"))

To proceed to search for sentinel images, we are going to use a map of urban areas that we have in the PERMANENT mapset. Since we are interested in the city of Cordoba, we use the function v.extract to create a new polygon corresponding to the municipality of Cordoba. Note that in this case we use `run_command()`.

In [ ]:
# Extract Cordoba urban area from `radios_urbanos`
gs.run_command("v.extract", input="radios_urbanos", 
               where="nombre == 'CORDOBA'", 
               output="radio_urbano_cba",
               overwrite=True)

We set the computational region to the boundaries of that vector. This will be the bounding box to be used for the sentinel image search.

In [ ]:
# Set the computational region to the extent of Cordoba urban area
region = gs.parse_command("g.region", vector="radio_urbano_cba", res=30, flags="g")
region.keys()

We now use the new `grass.jupyter` functions to display the newly obtained vector over the OpenStreetMap basemap.

In [ ]:
# Display newly created vector
cba_map = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
# cba_map.add_raster("elevation", opacity=0.5)
cba_map.add_vector("radio_urbano_cba")
cba_map.add_layer_control(position = "bottomright")
cba_map.show()

By way of illustration, we also show the 3D elevation map where the colors represent the land uses.

In [ ]:
# Display a 3d map
cba_3dmap = gj.Map3D(width=600, use_region=True, mode="fine")
cba_3dmap.render(elevation_map="elevation", color_map="landcover_2018", perspective=12)
cba_3dmap.show()

### Sentinel 2 data search and download

We installed the `i.sentinel` toolbox which consists of several modules that facilitate searching, filtering, downloading, importing and pre-processing Sentinel data, especially Sentinel 2, from a GRASS GIS session. See [i.sentinel](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.html) and the links to each module for more details. 

In [ ]:
# install extension
gs.run_command("g.extension", extension="i.sentinel")

We use the `i.sentinel.download` module which internally uses the `sentinelsat` library to search and filter scenes containing the defined region. For this we need to be registered in the Copernicus hub and have our credentials in a text file. For more details about the function and its uses, visit the [manual](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html).

In [ ]:
list_prod = gs.read_command("i.sentinel.download", settings=s2_credentials, 
                            start=start_date, end=end_date, 
                            producttype="S2MSI2A", clouds=30, 
                            area_relation="Contains", 
                            footprints="s2_footprints", 
                            flags="l")

In [ ]:
list_prod

We improved the previous output to make it easier to read and converted it into a data frame.

In [ ]:
lista = []
for line in list_prod.split("\n"):
    img = line.split(" ")
    if len(img) == 8:
        img.pop(3) # remove annoying empty space bc of padding for cloud percentage
    lista.append(img)

lista.pop(len(lista)-1) # remove last empty line

In [ ]:
# turn interim list into nice table
df_lista = pd.DataFrame(lista, columns=['uuid', 'scene', 'date', 'cloud', 'product', 'size', 'unit'])
df_lista

In [ ]:
# diplay footprints (you may want to zoom out a bit)
cba_map = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
cba_map.add_vector("s2_footprints")
cba_map.add_layer_control(position = "bottomright")
cba_map.show()

The next step is to download the scene or scenes of interest. For that we use the module without the `l` flag for list.

In [ ]:
# download of the selected scene (2022-03-25, T14:10:51Z)
gs.run_command("i.sentinel.download", settings=s2_credentials, 
               uuid="84f3695e-4ed8-4345-b2af-ba6d6d81af2d", 
               output=s2_data_folder)

In [ ]:
# import only bands relevant for RGB, NDVI and NDWI
gs.run_command("i.sentinel.import", 
               input=s2_data_folder, 
               pattern_file="*20220325T141051*",
               pattern="B(02_1|03_1|04_1|08_1|8A_2|11_2|12_2)0m", 
               extent="region", # subset import to region extent
               flags="rcsj") # reproject, clouds, shadow, json metadata

In [ ]:
# list raster maps
gs.list_grouped(type="raster")['sentinel2']

In [ ]:
# check metadata of some imported bands
gs.raster_info(map="T20JLL_20220325T141051_B03_10m")

A fairly new concept within GRASS is semantic labels. These are especially relevant for satellite imagery as they allow us to identify to which sensor and band a given raster corresponds. These labels are particularly relevant when working with satellite image collections and also when classifying different scenes. We will see it later, but by generating a spectral signature for a given set of bands, it can be re-used to classify another scene as long as the semantic labels are the same.

In [ ]:
# print semantic labels
for i in gs.list_grouped(type="raster")['sentinel2']:
    label = gs.raster_info(map=i)['semantic_label']
    print('Map: {}, Semantic label: {}'.format(i,label))

In [ ]:
# Create Map instance
b3_map = gj.Map(width=300)
# Add a raster, vector and legend to the map
b3_map.d_rast(map="T20JLL_20220325T141051_B03_10m")
# Display map
b3_map.show()

In [ ]:
# apply grey color to RGB bands
gs.run_command("r.colors", 
               map="T20JLL_20220325T141051_B04_10m,T20JLL_20220325T141051_B03_10m,T20JLL_20220325T141051_B02_10m",
               color="grey")

In [ ]:
# perform color auto-balancing for RGB bands
gs.run_command("i.colors.enhance", 
               red="T20JLL_20220325T141051_B04_10m",
               green="T20JLL_20220325T141051_B03_10m", 
               blue="T20JLL_20220325T141051_B02_10m",
               strength=90)

In [ ]:
# display the enhanced RGB combination
cba_rgb = gj.Map(use_region=True)
cba_rgb.d_rgb(red="T20JLL_20220325T141051_B04_10m", 
              green="T20JLL_20220325T141051_B03_10m", 
              blue="T20JLL_20220325T141051_B02_10m")
cba_rgb.show()

### Identification and masking of clouds

Since we will be starting to generate new raster maps, it is essential that we set the computational region to the limits and resolution of one of our bands. We may also be interested in a smaller area for initial testing. This is extremely easy and avoids having to cut raster to raster physically.

In [ ]:
gs.parse_command("g.region", raster="T20JLL_20220325T141051_B03_10m", flags="g")

We use the `i.sentinel.mask` module that takes the metadata recorded when importing bands.

In [ ]:
# identify and mask clouds and clouds shadows: i.sentinel.mask
gs.run_command("i.sentinel.mask",
               blue="T20JLL_20220325T141051_B02_10m",
               green="T20JLL_20220325T141051_B03_10m",
               red="T20JLL_20220325T141051_B04_10m", 
               nir="T20JLL_20220325T141051_B08_10m",
               nir8a="T20JLL_20220325T141051_B8A_20m",
               swir11="T20JLL_20220325T141051_B11_20m",
               swir12="T20JLL_20220325T141051_B12_20m",
               cloud_mask="cloud", 
               shadow_mask="shadow",
               scale_fac=10000, 
               flags="s", overwrite=True)

In [ ]:
# list vector maps in the mapset
gs.list_grouped(type="vector")['sentinel2']

In [ ]:
# display output
cba_rgb.d_vect(map="shadow", color="red", fill_color="red")
cba_rgb.d_vect(map="cloud", color="blue", fill_color="blue")
cba_rgb.show()

We inspect the cloud map downloaded with the data and set the same color palette for the purpose of comparing both products: the cloud and shadow mask obtained with i.sentinel.mask and the one provided by ESA.

In [ ]:
!v.db.select T20JLL_20220325T141051_MSK_CLOUDS | head

In [ ]:
s2_clouds = "T20JLL_20220325T141051_MSK_CLOUDS"
colours = ["1 0:0:255", "2 255:0:0"]
colourise = gs.feed_command("v.colors", map=s2_clouds, use="attr", column="value", rules="-", quiet=True)
colourise.stdin.write("\n".join(colours).encode())
colourise.stdin.close()

In [ ]:
cba_rgb = gj.Map(use_region=True)
cba_rgb.d_rgb(red="T20JLL_20220325T141051_B04_10m", 
              green="T20JLL_20220325T141051_B03_10m", 
              blue="T20JLL_20220325T141051_B02_10m")
cba_rgb.d_vect(map="T20JLL_20220325T141051_MSK_CLOUDS")
cba_rgb.show()

### Spectral indices of vegetation and water

Before proceeding to calculate the indices, let's mask the areas identified as clouds and shadows. To do this, we first paste the vectors into one vector and then apply it as an inverse mask. For more details on how the masks work in GRASS see [r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html)

In [ ]:
# set clouds mask
gs.run_command("v.patch", input="cloud,shadow", output="cloud_shadow_mask")
gs.run_command("r.mask", vector="cloud_shadow_mask", flags="i")

We then used i.vi and i.wi (addon) to estimate NDVI and NDWI. See [i.vi](https://grass.osgeo.org/grass-stable/manuals/i.vi.html) and [i.wi](https://grass.osgeo.org/grass-stable/manuals/addons/i.wi.html) for more details on the available indices.

In [ ]:
# estimate vegetation indices
gs.run_command("i.vi", 
               red="T20JLL_20220325T141051_B04_10m", 
               nir="T20JLL_20220325T141051_B08_10m", 
               output="T20JLL_20220325T141051_NDVI_10m", 
               viname="ndvi")

# add semantic label
gs.run_command("r.support", map="T20JLL_20220325T141051_NDVI_10m", semantic_label="S2_NDVI")

In [ ]:
ndvi_map = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
ndvi_map.add_raster("T20JLL_20220325T141051_NDVI_10m", opacity=0.7)
ndvi_map.add_layer_control(position = "bottomright")
ndvi_map.show()

In [ ]:
# install extension
gs.run_command("g.extension", extension="i.wi")

In [ ]:
# estimate water indices and set color palette
gs.run_command("i.wi", 
               green="T20JLL_20220325T141051_B04_10m",
               nir="T20JLL_20220325T141051_B08_10m",
               output="T20JLL_20220325T141051_NDWI_10m",
               winame="ndwi_mf")

# add semantic label
gs.run_command("r.support", map="T20JLL_20220325T141051_NDWI_10m", semantic_label="S2_NDWI")

# set ndwi color table
gs.run_command("r.colors", map="T20JLL_20220325T141051_NDWI_10m", color="ndwi")

In [ ]:
ndwi_map = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
ndwi_map.add_raster("T20JLL_20220325T141051_NDWI_10m", opacity=0.7)
ndwi_map.add_layer_control(position = "bottomright")
ndwi_map.show()

#### GRASS Maps as numpy arrays

GRASS maps can be read as numpy arrays thanks to the array function of the grass.script library. This facilitates many operations with python libraries that require an array as input. In this case, we demonstrate its use with a histogram.

In [ ]:
#Read NDVI as numpy array
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

ndvi = garray.array(mapname="T20JLL_20220325T141051_NDVI_10m", null='nan')
ndvi.shape


In [ ]:
sns.set_style('darkgrid')
sns.histplot(ndvi.ravel(), kde=False)

### Segmentation and clustering of segments

There are several modules to perform segmentation in GRASS GIS, the most known are [i.segment]() and [i.superpixels.slic](). We will demonstrate their use and then perform a KMeans clustering with the result of one of them.

In [ ]:
# list maps
s2_maps = gs.list_grouped(type="raster", pattern="*20220325T141051*")['sentinel2']
print(s2_maps)

When you work with stack of rasters in GRASS, you create what's called a group, which is a stack, but based on metadata, so it doesn't take up more space. 

In [ ]:
# create group and subgroup with bands and indices
gs.run_command("i.group", group="s2", subgroup="s2", input=s2_maps)
gs.parse_command("i.group", group="s2", flags="l")

#### Superpixels

In [ ]:
# install extension
gs.run_command("g.extension", extension="i.superpixels.slic")

In [ ]:
# run i.superpixels.slic
gs.run_command("i.superpixels.slic", input="s2", output="superpixels", num_pixels=2000)

# convert the resulting raster to vector
gs.run_command("r.to.vect", input="superpixels", output="superpixels", type="area")

#### Region growing

In [ ]:
# run i.segment (region growing)
gs.run_command("i.segment", group="s2", output="segments", threshold=0.5, minsize=50, memory=500)

# convert the resulting raster to vector
gs.run_command("r.to.vect", input="segments", output="segments", type="area")

We compare the number of segments obtained:

In [ ]:
# compare number of segments
n1=gs.vector_info(map="superpixels")['areas']
n2=gs.vector_info(map="segments")['areas']

print("Superpixels SLIC: {}; Region growing: {}".format(n1,n2))

In [ ]:
# diplay results
seg_map = gj.InteractiveMap(width = 600, use_region=True, tiles="OpenStreetMap")
seg_map.add_raster("T20JLL_20220325T141051_NDVI_10m", opacity=0.5)
#seg_map.add_vector("superpixels")
seg_map.add_vector("segments")
seg_map.add_layer_control(position = "bottomright")
seg_map.show()

Now, let's extract statistics for the segments from the bands of the group "s2"

In [ ]:
# extract statistics from the segments, make graphs and a clustering
gs.run_command("v.rast.stats", map="segments", raster=s2_maps, column_prefix=s2_maps, method="average")

In [ ]:
tabla_segs = pd.DataFrame.from_dict(gs.vector_db_select(map="segments")['values'], 
                                       orient='index', 
                                       columns=gs.vector_db_select(map="segments")['columns'])

In [ ]:
tabla_segs.head()

In [ ]:
# univar stats
gs.parse_command("v.univar", map="segments", column="T20JLL_20220325T141051_NDVI_10m_average", flags="g")

In [ ]:
# convert columns from type object to float
for col in tabla_segs.columns[3:]:
    tabla_segs[col] = tabla_segs[col].astype('float')

In [ ]:
fig = plt.figure(figsize=(15,10))
sns.boxplot(data=tabla_segs.iloc[:,3:10])
plt.show()

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.violinplot(data=tabla_segs.iloc[:,10:12])
plt.show()

### Clustering

In [ ]:
# convert dataframe to array
segs_arr = tabla_segs.iloc[:,10:12].to_numpy()
segs_arr

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(segs_arr)
y_kmeans = kmeans.predict(segs_arr)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
plt.scatter(segs_arr[:, 0], segs_arr[:, 1], c=y_kmeans, s=80, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

It could be said that there is not a good separation of groups according to NDVI and NDWI. This may be due to the segments themselves, their shape and size, as well as to the fact that more dimensions are needed to identify different groups; or indeed, that there are no such clearly distinguishable groups. 

In [ ]:
# how to attach the labels to the segments to show it spatially?

### Supervised Cassification: Maximum Likelihood

We will now demonstrate the workflow to perform a supervised maximum likelihood classification from digitized training polygons in GRASS.

In [ ]:
# import gpkg with training areas
gs.run_command("v.import", input=train_areas, output="training")

# use color column for classes
gs.run_command("v.colors", map="training", rgb_column="color", flags="c")

In [ ]:
cba_rgb_train = gj.Map(width=300, use_region=True)
cba_rgb_train.d_rgb(red="T20JLL_20220325T141051_B04_10m", 
              green="T20JLL_20220325T141051_B03_10m", 
              blue="T20JLL_20220325T141051_B02_10m")
cba_rgb_train.d_vect(map="training")
cba_rgb_train.show()

In [ ]:
# convert to raster
gs.run_command("v.to.rast", input="training", output="training", 
               use="attr", attribute_column="cat_", label_column="class")

In [ ]:
# obtain signature files
gs.run_command("i.gensig", trainingmap="training", group="s2", subgroup="s2", signaturefile="sig_sentinel")

In [ ]:
# perform ML supervised classification
gs.run_command("i.maxlik", group="s2", subgroup="s2", signaturefile="sig_sentinel", output="sentinel_maxlik")

In [ ]:
# label classes
label_class = ["1:vegetation", "2:urban", "3:bare soil"]
categorise = gs.feed_command("r.category", map="sentinel_maxlik", separator=":", rules="-", quiet=True)
categorise.stdin.write("\n".join(label_class).encode())
categorise.stdin.close()

In [ ]:
# display results
cba_sup_class = gj.Map(width=500, use_region=True)
cba_sup_class.d_rast(map="sentinel_maxlik")
cba_sup_class.d_legend(raster="sentinel_maxlik", title="Class", fontsize=10, at=(80, 93, 80, 90), flags="b")
cba_sup_class.d_barscale()
cba_sup_class.show()

In [ ]:
# percentage of each class
gs.parse_command("r.report", map="sentinel_maxlik", units="p", flags="h")

In [ ]:
# class statistics: NDVI
class_stats = gs.read_command("r.univar", map="T20JLL_20220325T141051_NDVI_10m", zones="sentinel_maxlik", flags="t")
class_stats_df = pd.DataFrame([line.split("|") for line in class_stats.splitlines()])
class_stats_df.columns = class_stats_df.iloc[0]


In [ ]:
df2 = class_stats_df.loc[1:,['label', 'min', 'max', 'mean']]
df2

In [ ]:
# remove former mask to avoid getting no data in the new scene
gs.run_command("r.mask", flags="r")

Next, and to demonstrate the use of semantic labels, we will classify another sentinel scene with the same signature obtained earlier. For this, we follow the same steps detailed at the beginning of this notebook.

In [ ]:
list_2022 = gs.read_command("i.sentinel.download", settings=s2_credentials, 
                            start="2022-02-01", end="2022-04-30", 
                            producttype="S2MSI2A", clouds=10, 
                            area_relation="Contains", 
                            flags="l")

lista2022 = []
for line in list_2022.split("\n"):
    img = line.split(" ")
    if len(img) == 8:
        img.pop(3) # remove annoying empty space bc of padding for cloud percentage
    lista2022.append(img)

lista2022.pop(len(lista2022)-1) # remove last empty line
pd.DataFrame(lista2022, columns=['uuid', 'scene', 'date', 'cloud', 'product', 'size', 'unit'])

In [ ]:
# download of the selected scene
gs.run_command("i.sentinel.download", settings=s2_credentials, 
               uuid="f31d3b36-a131-44e8-b7b3-880b6c8a40c4", 
               output=s2_data_folder, sleep=10)

In [ ]:
gs.run_command("i.sentinel.import", 
               input=s2_data_folder, 
               pattern_file="*20220328T141741*",
               pattern="B(02_1|03_1|04_1|08_1|8A_2|11_2|12_2)0m", 
               extent="region", # import only region extent
               flags="rcsj") # reproject, clouds, shadow, json metadata

In [ ]:
# apply grey color to RGB bands
gs.run_command("r.colors", 
               map="T20JLL_20220328T141741_B04_10m,T20JLL_20220328T141741_B03_10m,T20JLL_20220328T141741_B02_10m",
               color="grey")

# perform color auto-balancing for RGB bands
gs.run_command("i.colors.enhance", 
               red="T20JLL_20220328T141741_B04_10m",
               green="T20JLL_20220328T141741_B03_10m", 
               blue="T20JLL_20220328T141741_B02_10m",
               strength=90)

# display the enhanced RGB combination
cba_rgb = gj.Map(use_region=True)
cba_rgb.d_rgb(red="T20JLL_20220328T141741_B04_10m", 
              green="T20JLL_20220328T141741_B03_10m", 
              blue="T20JLL_20220328T141741_B02_10m")
cba_rgb.d_barscale()
cba_rgb.show()

In [ ]:
# ndvi
gs.run_command("i.vi", 
               red="T20JLL_20220328T141741_B04_10m", 
               nir="T20JLL_20220328T141741_B08_10m", 
               output="T20JLL_20220328T141741_NDVI_10m", 
               viname="ndvi")

# add semantic label
gs.run_command("r.support", map="T20JLL_20220328T141741_NDVI_10m", semantic_label="S2_NDVI")

# ndwi
gs.run_command("i.wi", 
               green="T20JLL_20220328T141741_B03_10m",
               nir="T20JLL_20220328T141741_B08_10m",
               output="T20JLL_20220328T141741_NDWI_10m",
               winame="ndwi_mf")

# add semantic label
gs.run_command("r.support", map="T20JLL_20220328T141741_NDWI_10m", semantic_label="S2_NDWI")

In [ ]:
s2_maps = gs.list_grouped(type="raster", pattern="*2022*")['sentinel2']
s2_maps

In [ ]:
gs.run_command("i.group", group="s2_2022", subgroup="s2_2022", input=s2_maps)
gs.parse_command("i.group", group="s2_2022", flags="l")

In [ ]:
gs.run_command("i.maxlik", group="s2_2022", subgroup="s2_2022", 
               signaturefile="sig_sentinel", output="sentinel_maxlik_2022")

In [ ]:
label_class = ["1:vegetation", "2:urban", "3:bare soil"]
categorise = gs.feed_command("r.category", map="sentinel_maxlik_2022", separator=":", rules="-", quiet=True)
categorise.stdin.write("\n".join(label_class).encode())
categorise.stdin.close()

In [ ]:
# display results
cba_sup_class_2022 = gj.Map(width=500, use_region=True)
cba_sup_class_2022.d_rast(map="sentinel_maxlik_2022")
cba_sup_class_2022.d_legend(raster="sentinel_maxlik_2022", title="Class", 
                            fontsize=10, at=(80, 93, 80, 90), flags="b")
cba_sup_class_2022.d_barscale()
cba_sup_class_2022.show()

In [ ]:
session.finish()

### References

- [GRASS GIS 8.2.0 Reference Manual](https://grass.osgeo.org/grass-stable/manuals/)
- [GRASS GIS Python library documentation](https://grass.osgeo.org/grass-stable/manuals/libpython/)
- https://github.com/ncsu-geoforall-lab/grass-gis-workshop-FOSS4G-2021
- https://github.com/wenzeslaus/python-grass-addon
- https://grass.osgeo.org/learn/tutorials/
- https://github.com/wenzeslaus/geospatial-modeling-course-jupyter